In [1]:
import numpy as np
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, interplevel, ll_to_xy)
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import xarray as xr
import gc, glob
import pickle
from tqdm import tqdm
import sys
sys.path.insert(1, '../../')
from util.wrf_process import (calc_derive, object_tracking, read_and_write, regrid)
import read_config

# Domain 2: No change

In [2]:
memb = 1
track = read_and_write.depickle(f'../../storage/memb0{memb}/proc/track.pkl')['trackobj']
LWstructre = read_and_write.depickle(f'../../storage/output/LWstructure.pkl')

In [24]:
np.swapaxes(np.asfortranarray(LWstructre,dtype=np.float64),0,1).T.tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/lws_d02.bin')

# Domain 1: Find closest point

In [3]:
# Open the NetCDF file
ncfile = Dataset(f'../../storage/memb0{memb}/wrfout_d02_2013-11-02_00:00:00')
# Get basic domain settings
lons,lats,pres = read_and_write.get_basic_domain(ncfile)
if (lons.min() < 0) and (lons.max() > 0):
    lon_offset = object_tracking.dateline_lon_shift(lons, reverse=0)
else:
    lon_offset = 0

In [4]:
domain='d02'
store,storeis = [],[]

if domain=='d02':
    #zcel = [i for i in range(len(track['latlon_xy']))]
    zcel = read_and_write.depickle(f'../../storage/memb0{memb}/proc/validindex.pkl') 
    loninterp = list(np.interp(np.linspace(0,zcel[-1],zcel[-1]*20), 
                               np.asarray(zcel), 
                               np.asarray([track['want_latlon'][i][0] for i in range(len(track['want_latlon']))][:int(zcel[-1]+1)])))
    latinterp = list(np.interp(np.linspace(0,zcel[-1],zcel[-1]*20), 
                               np.asarray(zcel), 
                               np.asarray([track['want_latlon'][i][1] for i in range(len(track['want_latlon']))][:int(zcel[-1]+1)])))
    track = object_tracking.tree_latlon_to_xy(ncfile,
                                                 (lons + lon_offset).data,
                                                 (lats).data,
                                                 loninterp,
                                                 latinterp,
                                                )
elif domain=='d01':
    zcel = read_and_write.depickle(f'../../storage/memb0{memb}/proc/validindex.pkl')
    loninterp = list(np.interp(np.linspace(0,zcel[-1],zcel[-1]*4), 
                               np.asarray(zcel), 
                               np.asarray([track['want_latlon'][i][0] for i in range(len(track['want_latlon']))][:int(zcel[-1]+1)])))
    latinterp = list(np.interp(np.linspace(0,zcel[-1],zcel[-1]*4), 
                               np.asarray(zcel), 
                               np.asarray([track['want_latlon'][i][1] for i in range(len(track['want_latlon']))][:int(zcel[-1]+1)])))
    trackd01 = object_tracking.tree_latlon_to_xy(ncfile,
                                                 (lons + lon_offset).data,
                                                 (lats).data,
                                                 loninterp,
                                                 latinterp,
                                                )
    
#for i in tqdm(range(len(latinterp))):#zcel:
#    if domain=='d01':
#        sizes0,sizes1,dze = 239,389,35
#        cntX,cntY = trackd01['latlon_xy'][i]
#    elif domain=='d02':
#        sizes0,sizes1,dze = 740,1200,175
#        cntX,cntY = track['latlon_xy'][i]
#    TEST = np.zeros((54,sizes0,sizes1))
#    TEST = np.zeros((54,sizes0,sizes1))
#    try:
#        for j in range(54):
#            TEST[j,cntY-dze:cntY+dze,cntX-dze:cntX+dze] = LWstructre[j,...]
#        farray = np.swapaxes(np.asfortranarray(TEST,dtype=np.float64),0,1)
#        farray.T.tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/lwpert{i+1}.bin')
#        del TEST,farray
#        gc.collect()
#        #store.append(TEST)
#        storeis.append(i)
#    except ValueError:
#        continue

In [7]:
domain='d01'
if domain=='d01':
    TEST = np.zeros((239,54,389),order='F',dtype=np.float64)
elif domain=='d02':
    TEST = np.zeros((740,54,1200),order='F',dtype=np.float64)
TEST.T.tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/lwpert0.bin')

# Output Tracks

In [15]:
trackX,trackY = [],[]
for itime in range(zcel[-1]*20):
    trackX.append(track['latlon_xy'][itime][0])
    trackY.append(track['latlon_xy'][itime][1])

In [16]:
np.array(trackX,dtype=np.int64).tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/trackX.bin')
np.array(trackY,dtype=np.int64).tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/trackY.bin')

In [10]:
np.array(trackX,dtype=np.int64).shape,(zcel[-1])

((1340,), 67)

# Output Time Stamps

In [17]:
import juliandate as jd
import datetime,time

def create_juliandates(start_time=None,ref_time=None,totaltimesteps=68*4,minutesdelta=15):
    savetimes = [start_time]
    for i in range(totaltimesteps):#20):
        start_time += datetime.timedelta(minutes=minutesdelta)
        savetimes.append(start_time)
    del i
    gc.collect()
    
    juliandates = []
    for i in range(len(savetimes)):
        mt,reft = savetimes[i],ref_time
        temp = jd.from_gregorian(mt.year,mt.month,mt.day,mt.hour,mt.minute)-\
        jd.from_gregorian(reft.year,reft.month,reft.day,reft.hour,reft.minute)
        juliandates.append(temp)
    return juliandates

In [18]:
domain='d02'
juliandates = create_juliandates(datetime.datetime.strptime('02/11/13 00:00:00', '%d/%m/%y %H:%M:%S'),
                   datetime.datetime.strptime('01/01/13 00:00:00', '%d/%m/%y %H:%M:%S'),
                   67*20,
                   3)
juliandates_arr = np.array(juliandates,dtype=np.float64)
#np.asfortranarray(juliandates).T.tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/juliandates.bin')

In [21]:
juliandates_arr[:-1].tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/juliandates.bin')

In [16]:
68*4+1

273

In [22]:
for i in range(len(store)):
    farray = np.swapaxes(np.asfortranarray(store[i]),0,1)#.tofile(f'../../storage/memb0{memb}/proc/lwperturb_d01.dat')
    farray.T.tofile(f'../../storage/memb0{memb}/proc/lwperturb_{str(domain)}/lwpert{i+1}.bin')

In [13]:
memb=1
A = np.fromfile(f'../../storage/memb0{memb}/proc/lwperturb_d02.bin')